** This file clean the tweets and make it ready for analysis based on location**

** It saves the whole process in a json file **

In [6]:
from config_file import DOWNLOAD_ROOT, DATASETS_PATH, DOWNLOAD_URL, TWEETS_PATH
from utils import *
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import string
import unicodedata
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
import re
from pyspark.conf import SparkConf
from pyspark.sql import functions as F
import string
import csv
from pyspark.sql.types import StringType
from pyspark.sql.types import ArrayType
import tarfile
from six.moves import urllib
from config_file import DOWNLOAD_ROOT, DATASETS_PATH, DOWNLOAD_URL, TWEETS_PATH

In [7]:
spark = SparkSession.builder.master("local[*]").\
        appName("nlp").config("spark.executor.memory", "32g").\
        config("spark.driver.memory", "32g").\
        config("spark.memory.offHeap.enabled",True).\
        config("spark.memory.offHeap.size","16g").\
        config("spark.debug.maxToStringFields","200").\
        getOrCreate()

In [4]:
if __name__ == '__main__':
    extract_tar_file(tgz_name = "datasets.tar.gz")
    tweets = load_data(TWEETS_PATH, '*.csv')
    tweets = tweets.selectExpr( "_c1 as text" , "_c12 as location")
    latitude_longitude= load_data(DATASETS_PATH, 'country_capitals.csv')
    latitude_longitude= latitude_longitude.select('countryName','capitalLatitude','capitalLongitude')
    
    udfhttps=udf(lambda text: remove_https(text), StringType())
    tweets = tweets.withColumn("text", udfhttps(tweets["text"]))
    split_col = split(tweets['location'], ':')
    tweets = tweets.withColumn('location_name', split_col.getItem(1))
    
    udfNormalizeData=udf(lambda text: normalizeData(text), StringType())
    tweets = tweets.withColumn("text", udfNormalizeData(tweets["text"]))
    tweets = tweets.withColumn("location_name", udfNormalizeData(tweets["location_name"]))
    
    udfDecoding=udf(lambda text: clean_latin(text), StringType())
    tweets = tweets.withColumn("text", udfDecoding(tweets["text"]))
    tweets= lower_words(tweets)
    tweet= tweets.where(tweets.location.isNotNull())
    tweets = tweets.drop('location')
    tweets= regexTokenizer_StopWordsRemover(tweets, 'text', 'words_','words')
    tweets = remove_emoji(tweets, "words","words_")

    regexTokenizer = RegexTokenizer(minTokenLength= 3, inputCol="location_name",outputCol="location_name_token", pattern="\\W")

    tweets = regexTokenizer.transform(tweets)
    tweets= tweets.where(size(col("location_name_token")) >0)
    join_udf = udf(lambda x: " ".join(x))
    tweets=tweets.withColumn("location_name_token", join_udf(col("location_name_token")))
    tweets = tweets.drop('text', 'location_name')
    tweets=tweets.selectExpr('location_name_token as location', 'words as words' )
    tweets = change_city_with_country(tweets)
    tweets.repartition(1).write.json(os.path.join(DATASETS_PATH,'tweets_location'))
    tweets_location = spark.read.json(os.path.join(DATASETS_PATH, "tweets_location", "tweets_location.json"))
    df_countries_happiness = avg_happiness_of_words(tweets_location, 'words')
    df.repartition(1).write.json(os.path.join(DATASETS_PATH,'countries_happiness'))


In [5]:
tweets.show(2)

+--------+--------------------+
|location|               words|
+--------+--------------------+
|  london|[game, last, nigh...|
|    none|[fifaworldcup, ng...|
+--------+--------------------+
only showing top 2 rows

